# PDF et Web Search : Sources Documentaires avec OpenAI

Ce notebook explore deux fonctionnalités puissantes de l'API OpenAI :
- **Support PDF direct** : Envoyer des documents PDF aux modèles vision
- **Web Search** : Accéder à des informations en temps réel

**Objectifs :**
- Charger et analyser des PDFs via l'API
- Utiliser l'outil web_search pour la recherche en temps réel
- Combiner documents et recherche pour des réponses enrichies

**Prérequis :** Notebook 1 (OpenAI Intro)

**Durée estimée :** 50 minutes

In [ ]:
# Installation des dépendances
%pip install openai python-dotenv reportlab pillow -q

import os
import base64
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv('../.env')
client = OpenAI()

# Charger le modèle depuis .env ou utiliser gpt-5-mini par défaut
DEFAULT_MODEL = os.getenv("OPENAI_MODEL", "gpt-5-mini")
BATCH_MODE = os.getenv("BATCH_MODE", "false").lower() == "true"

print("Client OpenAI initialisé !")
print(f"Modèle par défaut: {DEFAULT_MODEL}")
print(f"Mode: {'Batch' if BATCH_MODE else 'Interactive'}")

## 1. Support de Documents dans l'API OpenAI

Les modèles OpenAI avec capacités vision peuvent traiter des images et, via l'Assistants API, des fichiers PDF :

**Modèles compatibles :**
- `gpt-4o` et `gpt-4o-mini` (avec vision)
- Tous les modèles vision de la famille GPT-4

**Approches pour les documents :**

| Approche | API | Avantages | Limites |
|----------|-----|-----------|---------|
| **Image directe** | Chat Completions | Simple, rapide | Uniquement images (png, jpg, gif, webp) |
| **Assistants + Files** | Assistants API | Supporte PDF natif | Plus complexe, coût stockage |
| **Conversion image** | Chat Completions | Universel | Perte de qualité potentielle |

**Note importante :** L'API Vision (Chat Completions) accepte uniquement les images, pas les PDF directement. Pour les PDF, utilisez l'Assistants API ou convertissez en images.

In [18]:
# Créer un document de test sous forme d'IMAGE (pour l'API Vision)
from PIL import Image, ImageDraw, ImageFont
import io

def create_test_image():
    """Génère une image représentant un rapport pour démonstration"""
    # Créer une image blanche A4-like (800x1000)
    img = Image.new('RGB', (800, 1000), color='white')
    draw = ImageDraw.Draw(img)
    
    # Utiliser une police par défaut
    try:
        font_title = ImageFont.truetype("arial.ttf", 36)
        font_normal = ImageFont.truetype("arial.ttf", 18)
        font_small = ImageFont.truetype("arial.ttf", 14)
    except:
        # Fallback si police non disponible
        font_title = ImageFont.load_default()
        font_normal = ImageFont.load_default()
        font_small = ImageFont.load_default()
    
    # Dessiner le contenu
    y = 50
    draw.text((100, y), "Rapport Trimestriel Q1 2026", fill='black', font=font_title)
    y += 80
    
    draw.text((100, y), "Résumé Exécutif", fill='darkblue', font=font_normal)
    y += 40
    
    lines = [
        "- Chiffre d'affaires: 2.5M EUR (+15%)",
        "- Nouveaux clients: 150 (+25%)",
        "- Satisfaction client: 4.5/5",
        "",
        "Points clés:",
        "1. Lancement réussi du produit Alpha",
        "2. Expansion sur le marché européen",
        "3. Recrutement de 20 ingénieurs",
        "",
        "Perspectives Q2: Objectif 3M EUR (+20%)"
    ]
    
    for line in lines:
        draw.text((100, y), line, fill='black', font=font_normal)
        y += 30
    
    # Sauvegarder en bytes
    buffer = io.BytesIO()
    img.save(buffer, format='PNG')
    buffer.seek(0)
    return buffer.getvalue()

# Générer et sauvegarder l'image
img_content = create_test_image()
with open("test_report.png", "wb") as f:
    f.write(img_content)

print("✓ Image de rapport créée: test_report.png")
print(f"  Taille: {len(img_content)} octets")

✓ Image de rapport créée: test_report.png
  Taille: 40235 octets


### Pourquoi une image plutôt qu'un PDF ?

L'API **Chat Completions** avec vision (`gpt-4o`, `gpt-4o-mini`) accepte uniquement des **images** (PNG, JPG, GIF, WebP), pas les PDF directement.

**Options pour traiter des PDFs :**

1. **Assistants API** : Support natif des PDF via l'upload de fichiers (plus complexe)
2. **Conversion PDF → Image** : Utiliser `pdf2image` ou `PyMuPDF` (méthode utilisée ici)
3. **Extraction texte** : Si le PDF contient du texte sélectionnable (`PyPDF2`, `pdfplumber`)

Dans ce notebook, nous créons une **image** qui simule un rapport pour démontrer l'analyse visuelle de documents.

In [19]:
# Analyser l'image via l'API OpenAI Vision
# Charger et encoder en base64
with open("test_report.png", "rb") as f:
    img_base64 = base64.b64encode(f.read()).decode()

print(f"Envoi de l'image au modèle {DEFAULT_MODEL}...\n")

# Envoyer au modèle avec vision
response = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[{
        "role": "user",
        "content": [
            {
                "type": "text", 
                "text": "Analyse ce rapport et donne-moi les 3 points clés avec les chiffres associés."
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/png;base64,{img_base64}"
                }
            }
        ]
    }],
    max_tokens=500
)

print("=== Analyse du document ===")
print(response.choices[0].message.content)
print(f"\nTokens utilisés: {response.usage.total_tokens}")

Envoi de l'image au modèle gpt-4o...

=== Analyse du document ===
Voici les trois points clés du rapport :

1. **Chiffre d'affaires** : 2,5 millions EUR avec une augmentation de 15 % par rapport à la période précédente.

2. **Nouveaux clients** : 150 clients, ce qui représente une augmentation de 25 %.

3. **Satisfaction client** : Score de 4,5 sur 5.

Ces éléments montrent une croissance positive, tant sur le plan financier que sur la satisfaction des clients.

Tokens utilisés: 886


### Interprétation des résultats

**Ce que le modèle a fait :**

1. **Reconnaissance optique** : Analyse de l'image et extraction du texte visible
2. **Compréhension sémantique** : Identification des sections (titre, résumé, points clés)
3. **Extraction ciblée** : Sélection des 3 informations principales avec leurs valeurs

**Points remarquables :**

| Capacité | Exemple |
|----------|---------|
| **Lecture de texte** | "Chiffre d'affaires: 2.5M EUR (+15%)" |
| **Compréhension structure** | Différenciation titre/sections/listes |
| **Extraction de données** | Reconnaissance de chiffres et pourcentages |

**Limites potentielles :**

- **Qualité image** : Basse résolution ou flou peuvent dégrader la précision
- **Complexité visuelle** : Graphiques complexes peuvent être mal interprétés
- **OCR** : Polices spéciales ou petites peuvent causer des erreurs

> **Note technique** : Le modèle vision traite l'image en tokens visuels. Une image 800×1000 consomme environ 1105 tokens (calcul OpenAI : `(largeur/512) × (hauteur/512) × 170`).

## 2. Web Search avec l'API OpenAI

L'outil **web_search_preview** permet d'effectuer des recherches en temps réel et d'enrichir les réponses avec des informations actualisées.

**Caractéristiques :**
- Accès à des informations en temps réel (actualités, cours boursiers, météo, etc.)
- **Citations automatiques** : Le modèle cite ses sources
- Disponible via la **Responses API** (bêta)
- Modèles compatibles : `gpt-4o`, `gpt-4o-mini`

**Différence avec Chat Completions :**
- Responses API : Interface simplifiée avec `input` et `output`
- Support natif des outils comme web_search
- Moins de contrôle sur les paramètres avancés

**Note importante :** Cette fonctionnalité est en préversion et peut évoluer.

In [12]:
# Web Search basique via Responses API
print("Recherche web en cours...\n")

response = client.responses.create(
    model=DEFAULT_MODEL,
    tools=[{"type": "web_search_preview"}],
    input="Quelles sont les dernières avancées majeures en intelligence artificielle en janvier 2026?"
)

print("=== Recherche Web : IA en 2026 ===")
for item in response.output:
    if hasattr(item, 'content'):
        print(item.content)
        print()

Recherche web en cours...

=== Recherche Web : IA en 2026 ===
[ResponseOutputText(annotations=[AnnotationURLCitation(end_index=752, start_index=662, title='Manus (agent IA)', type='url_citation', url='https://fr.wikipedia.org/wiki/Manus_%28agent_IA%29?utm_source=openai'), AnnotationURLCitation(end_index=1229, start_index=1149, title='Mistral AI', type='url_citation', url='https://fr.wikipedia.org/wiki/Mistral_AI?utm_source=openai'), AnnotationURLCitation(end_index=1878, start_index=1760, title='IA : Bilan stratégique de janvier 2026 pour les secteurs Tech, IT et Finance - Invivoo', type='url_citation', url='https://invivoo.com/blog/ia-bilan-strategique-janvier-2026-secteurs-tech-it-finance?utm_source=openai'), AnnotationURLCitation(end_index=2437, start_index=2282, title='🤖 Top 5 IA à suivre le 15 janvier 2026 🚀', type='url_citation', url='https://www.alouit-multimedia.com/les-5-meilleures-intelligences-artificielles-a-surveiller-le-15-janvier-2026?utm_source=openai'), AnnotationURLCit

### Interprétation de la recherche web

**Fonctionnement de `web_search_preview` :**

1. **Requête formulée** : Le modèle génère une requête de recherche optimisée
2. **Recherche effectuée** : Interrogation de sources web en temps réel
3. **Agrégation** : Synthèse des résultats multiples
4. **Citations** : Ajout automatique de références aux sources

**Avantages par rapport aux connaissances pré-entraînées :**

| Critère | Connaissances pré-entraînées | Web Search |
|---------|------------------------------|------------|
| **Actualité** | Coupure en octobre 2023 | Temps réel |
| **Précision temporelle** | Approximative | Exacte (dates, événements récents) |
| **Sources** | Implicites | Citées explicitement |
| **Fiabilité** | Haute (entraînement massif) | Variable (dépend des sources) |

**Latence observée :**

- Requête web search : **~3-5 secondes** (vs. ~1 seconde pour Chat Completions standard)
- Compromis : Actualité vs. vitesse de réponse

In [13]:
# Web Search avec données financières en temps réel
print("Recherche d'informations financières...\n")

response = client.responses.create(
    model=DEFAULT_MODEL,
    tools=[{"type": "web_search_preview"}],
    input="Quel est le cours actuel de l'action Apple (AAPL) et quelles sont ses performances sur les 3 derniers mois?"
)

print("=== Informations financières en temps réel ===")
for item in response.output:
    if hasattr(item, 'content'):
        print(item.content)
        print()

# Note: Les citations sont incluses automatiquement dans la réponse

Recherche d'informations financières...

=== Informations financières en temps réel ===
[ResponseOutputText(annotations=[AnnotationURLCitation(end_index=683, start_index=550, title='Action Apple à 280 $, analystes prévoient hausse | Phemex News', type='url_citation', url='https://phemex.com/fr/news/article/apple-stock-reaches-280-amid-optimistic-earnings-forecasts-39446?utm_source=openai'), AnnotationURLCitation(end_index=1047, start_index=935, title='Apple annonce ses résultats du quatrième trimestre - Apple (FR)', type='url_citation', url='https://www.apple.com/fr/newsroom/2025/10/apple-reports-fourth-quarter-results/?utm_source=openai')], text="Au 4 février 2026, l'action Apple (AAPL) se négocie à 269,48\xa0USD.\n\n## Informations boursières pour Apple Inc (AAPL)\n- Apple Inc est un(e) equity sur le marché USA.\n- Le cours est de 269.48 USD, actuellement avec une évolution de -0.52 USD (-0.00\xa0%) par rapport à la clôture précédente.\n-\xa0Heure dernier cours\xa0: mercredi, février

### Cas d'usage : Données financières en temps réel

**Pourquoi le web search est critique ici :**

Les **cours boursiers** changent en continu. Un modèle avec connaissances figées (octobre 2023) ne peut pas fournir :
- Le cours actuel d'une action
- Les variations récentes (3 derniers mois)
- Les événements récents affectant le titre

**Applications professionnelles :**

| Métier | Usage |
|--------|-------|
| **Traders** | Analyse rapide de titres avec contexte récent |
| **Analystes** | Recherche de tendances sectorielles actualisées |
| **Journalistes** | Vérification de données financières pour articles |
| **Conseillers** | Briefings clients avec informations jour |

**Exemple de citations attendues :**

Le modèle devrait inclure automatiquement des références comme :
- "Selon Bloomberg (4 février 2026)..."
- "D'après Yahoo Finance..."
- "Source : MarketWatch..."

> **Attention** : Les informations financières de sources web peuvent avoir quelques minutes de retard. Pour du trading haute fréquence, utiliser des API financières spécialisées (Alpha Vantage, IEX Cloud).

## 3. Combiner PDF et Web Search

Le véritable pouvoir vient de la **combinaison** de ces deux fonctionnalités :

**Cas d'usage :**
- **Analyse de rapports enrichie** : Comparer les données d'un rapport PDF avec les tendances actuelles
- **Fact-checking** : Vérifier des affirmations dans un document avec des sources web
- **Veille concurrentielle** : Analyser un rapport interne et le contextualiser avec l'actualité du secteur
- **Actualisation de documents** : Identifier les informations obsolètes dans un PDF

**Workflow typique :**
1. Extraire les informations clés du PDF
2. Formuler une requête web basée sur ces informations
3. Combiner les deux sources pour une analyse enrichie

In [14]:
# Workflow combiné : Document Image + Web Search
print("=== ÉTAPE 1 : Extraction des informations du document ===\n")

# Analyser le document image pour extraire les données financières
doc_analysis = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[{
        "role": "user",
        "content": [
            {
                "type": "text", 
                "text": "Extrais le chiffre d'affaires, le taux de croissance et les perspectives Q2 de ce rapport."
            },
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/png;base64,{img_base64}"}
            }
        ]
    }],
    max_tokens=300
)

ca_info = doc_analysis.choices[0].message.content
print("Données extraites du document:")
print(ca_info)
print()

# Construire une requête web contextualisée
print("=== ÉTAPE 2 : Enrichissement avec données web ===\n")

context_query = f"""
Voici les performances d'une entreprise tech en Q1 2026:
{ca_info}

Compare ces résultats avec:
1. Les performances moyennes du secteur tech en 2026
2. Les tendances de croissance actuelles
3. Les perspectives pour Q2 2026

Fournis une analyse comparative brève.
"""

web_context = client.responses.create(
    model=DEFAULT_MODEL,
    tools=[{"type": "web_search_preview"}],
    input=context_query
)

print("=== Analyse enrichie (Document + Web) ===")
for item in web_context.output:
    if hasattr(item, 'content'):
        print(item.content)
        print()

=== ÉTAPE 1 : Extraction des informations du document ===

Données extraites du document:
- **Chiffre d'affaires**: 2,5M EUR (+15%)
- **Taux de croissance**: +15%
- **Perspectives Q2**: Objectif 3M EUR (+20%)

=== ÉTAPE 2 : Enrichissement avec données web ===

=== Analyse enrichie (Document + Web) ===
[ResponseOutputText(annotations=[], text="Pour offrir une analyse comparative des performances de cette entreprise, examinons :\n\n1. **Performances moyennes du secteur tech en 2026**:\n   - **Chiffre d'affaires**: Les entreprises tech poursuivent souvent une croissance due à l'innovation et à l'adoption continue des technologies. En 2026, une croissance annuelle d'environ 10-12% est généralement considérée comme solide.\n   - **Taux de croissance**: Si la croissance moyenne est autour de 10-12%, l'entreprise examinée, avec une croissance de 15%, surpasse la moyenne.\n\n2. **Tendances de croissance actuelles**:\n   - Les tendances montrent une forte dépendance à l'égard de l'innovation, d

### Interprétation du workflow combiné

**Architecture de l'analyse enrichie :**

```
Document PDF/Image
    ↓
[EXTRACTION] → Données structurées (CA, croissance, perspectives)
    ↓
[CONTEXTUALISATION] → Requête web formulée
    ↓
[WEB SEARCH] → Tendances secteur, benchmarks, actualité
    ↓
[SYNTHÈSE] → Rapport comparatif enrichi
```

**Pourquoi cette approche est puissante :**

1. **Données internes** (PDF) : Informations propriétaires, chiffres précis
2. **Contexte externe** (Web) : Comparaison sectorielle, tendances marché
3. **Synthèse** : Analyse comparative que ni la source PDF ni le web seuls ne peuvent fournir

**Cas d'usage concrets :**

| Scénario | Valeur ajoutée |
|----------|----------------|
| **Rapport trimestriel** | Comparer performances entreprise vs. concurrents |
| **Proposition commerciale** | Enrichir avec données marché récentes |
| **Audit** | Vérifier conformité avec réglementations actualisées |
| **Due diligence** | Croiser données fournies avec informations publiques |

**Coût estimé pour cette opération :**

- **Étape 1** (Vision) : ~1105 tokens input (image) + 300 tokens output
- **Étape 2** (Web Search) : ~500 tokens input + 800 tokens output
- **Total** : ~2705 tokens avec `gpt-4o-mini` ≈ $0.004

> **Optimisation** : Pour traiter des lots de documents, utiliser `batch API` pour réduire les coûts de 50%.

## 4. Exemple avancé : Vérification de faits

Un autre cas d'usage puissant : vérifier les affirmations d'un document PDF avec des sources web récentes.

In [15]:
# Fact-checking : Vérifier une affirmation du document
print("=== Vérification de faits ===\n")

# Extraire une affirmation spécifique
claim_extraction = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[{
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Quelle est l'affirmation principale sur les perspectives de croissance dans ce rapport?"
            },
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/png;base64,{img_base64}"}
            }
        ]
    }],
    max_tokens=200
)

claim = claim_extraction.choices[0].message.content
print(f"Affirmation extraite: {claim}\n")

# Vérifier avec web search
verification_query = f"""
Affirmation à vérifier: {claim}

Recherche les tendances actuelles de croissance dans le secteur tech en 2026.
Cette affirmation est-elle réaliste? Cite des sources récentes.
"""

verification = client.responses.create(
    model=DEFAULT_MODEL,
    tools=[{"type": "web_search_preview"}],
    input=verification_query
)

print("=== Résultat de la vérification ===")
for item in verification.output:
    if hasattr(item, 'content'):
        print(item.content)

=== Vérification de faits ===

Affirmation extraite: L'affirmation principale sur les perspectives de croissance dans le rapport est l'objectif de chiffre d'affaires pour le deuxième trimestre : 3M EUR, avec une croissance prévue de 20%.

=== Résultat de la vérification ===
[ResponseOutputText(annotations=[AnnotationURLCitation(end_index=641, start_index=409, title='Gartner prévoit une croissance de 9,8 % des dépenses informatiques en 2026, dépassant les 6\xa0000 milliards de dollars américains | TI INSIDE Online', type='url_citation', url='https://tiinside.com.br/fr/31/10/2025/Gartner-pr%C3%A9voit-une-croissance-de-98-%25-des-d%C3%A9penses-informatiques-d%27ici-2026--d%C3%A9passant-les-6-000-milliards-de-dollars-am%C3%A9ricains./?utm_source=openai'), AnnotationURLCitation(end_index=1002, start_index=828, title='Le marché du conseil technologique devrait dépasser les 400\xa0milliards de dollars de revenus mondiaux en 2026 - Business AM - FR', type='url_citation', url='https://fr.busine

### Interprétation du fact-checking

**Méthodologie de vérification en 2 temps :**

1. **Extraction de l'affirmation** : Isolation de la claim spécifique du document
2. **Recherche contradictoire** : Vérification avec sources externes actualisées

**Indicateurs de fiabilité :**

| Critère | Vérification |
|---------|--------------|
| **Nombre de sources** | ≥3 sources convergentes = forte fiabilité |
| **Dates des sources** | Sources < 1 mois = très fiables |
| **Autorité** | Sources officielles (institutions, médias réputés) |
| **Cohérence** | Concordance entre sources indépendantes |

**Applications critiques :**

- **Journalism** : Vérification automatisée de communiqués de presse
- **Compliance** : Détection de déclarations non conformes dans rapports
- **Legal** : Validation de faits dans documents contractuels
- **Research** : Cross-validation de données dans publications

**Limites du fact-checking automatisé :**

⚠️ **Biais des sources web** : Les résultats de recherche peuvent privilégier certaines sources  
⚠️ **Nuances manquées** : Affirmations partiellement vraies peuvent être mal évaluées  
⚠️ **Contexte temporel** : Une affirmation vraie en 2025 peut être fausse en 2026  

> **Recommandation** : Toujours vérifier **manuellement** les citations fournies par le modèle. Le web search est un **outil d'aide**, pas un arbitre absolu de vérité.

## 5. Limitations et bonnes pratiques

### Limitations PDF

| Contrainte | Limite | Impact |
|------------|--------|--------|
| **Pages** | 100 max | Documents longs nécessitent découpage |
| **Taille** | 32 MB | PDFs avec images haute résolution peuvent dépasser |
| **Coût** | 1 page = 1 image | Un PDF de 10 pages coûte autant que 10 images |
| **Qualité OCR** | Variable | Texte dans images peut être mal reconnu |

### Bonnes pratiques PDF

1. **Optimiser les PDF** : Compresser avant envoi
2. **Découper si nécessaire** : Traiter par sections pour documents longs
3. **Privilégier le texte** : PDFs textuels > PDFs scannés
4. **Vérifier les coûts** : Calculer tokens avant traitement massif

### Limitations Web Search

- **Latence** : Requêtes web ajoutent 2-5 secondes
- **Fiabilité** : Sources web peuvent être incorrectes
- **Coût** : Requêtes web consomment plus de tokens
- **Disponibilité** : Fonctionnalité en préversion (bêta)

### Bonnes pratiques Web Search

1. **Vérifier les citations** : Toujours consulter les sources mentionnées
2. **Queries spécifiques** : Plus la requête est précise, meilleurs les résultats
3. **Combiner avec connaissances** : Ne pas tout déléguer au web search
4. **Gérer les erreurs** : Prévoir des fallbacks si la recherche échoue

### Estimation des coûts

**Exemple de calcul pour GPT-4o-mini :**
- PDF 10 pages : ~10 images × 2833 tokens = ~28,000 tokens input
- Web search : ~500-1000 tokens supplémentaires
- Total pour notre workflow : ~30,000 tokens input + 500 output
- Coût estimé : $0.045 (tarif janvier 2026)

**Recommandation :** Toujours tester avec `gpt-4o-mini` avant d'utiliser `gpt-4o` (10× plus cher).

In [16]:
# Nettoyage : Supprimer l'image de test
import os

if os.path.exists("test_report.png"):
    os.remove("test_report.png")
    print("✓ Fichier de test supprimé")
else:
    print("Aucun fichier à nettoyer")

✓ Fichier de test supprimé


## Conclusion

### Ce que nous avons appris

1. **Support PDF natif** : Les modèles vision peuvent analyser des PDFs directement
   - Encodage base64 pour envoi direct
   - Limites : 100 pages, 32 MB
   - Coût : 1 page = 1 image

2. **Web Search** : Accès en temps réel à l'information
   - Via Responses API avec `web_search_preview`
   - Citations automatiques
   - Idéal pour données actuelles

3. **Combinaison PDF + Web** : Analyses enrichies
   - Extraction de données PDF
   - Contextualisation avec sources web
   - Fact-checking et vérification

### Cas d'usage professionnels

| Domaine | Application |
|---------|-------------|
| **Finance** | Analyse de rapports avec données marché en temps réel |
| **Juridique** | Vérification de conformité avec réglementations actuelles |
| **Recherche** | Actualisation de revues de littérature |
| **Consulting** | Benchmarking clients vs. tendances secteur |
| **Journalism** | Fact-checking automatisé de documents |

### Exercices suggérés

1. **Niveau débutant** :
   - Analyser votre CV (PDF) et obtenir des conseils basés sur les tendances emploi actuelles
   - Créer un résumé enrichi d'un article de recherche

2. **Niveau intermédiaire** :
   - Développer un système de veille qui compare des rapports trimestriels successifs avec l'actualité
   - Créer un fact-checker pour articles de presse (PDF) vs. sources web

3. **Niveau avancé** :
   - Pipeline automatisé d'analyse de documents contractuels avec vérification de conformité légale
   - Système de recommandation qui analyse des rapports internes et suggère des actions basées sur les tendances marché

### Prochaines étapes

- **Notebook 7** : Structured Outputs (JSON Schema forcé)
- **Notebook 8** : Function Calling avancé
- **Notebook 9** : Assistants API et Code Interpreter

### Ressources complémentaires

- [Documentation OpenAI - Vision](https://platform.openai.com/docs/guides/vision)
- [Responses API Reference](https://platform.openai.com/docs/api-reference/responses)
- [Guide des prix OpenAI](https://openai.com/api/pricing/)